# LLaMA-Factory微调Qwen3-4B-BASE

使用AutoDL vGPU-32GB 进行实验，已预装CUDA

环境为
torch=2.3.0
python=3.12
ubuntu22.04
cuda_version=12.1

## 1.安装LLaMA-Factory库及相关依赖（官方安装方法）
```bash
git clone --depth 1 https://github.com/hiyouga/LLaMA-Factory.git
cd LLaMA-Factory
pip install -e ".[torch,metrics]" --no-build-isolation
```

### 检查LLaMA-Factory安装

#### 检查版本号
```bash
llamafactory-cli version
```

### 安装bitsandtypes

用于启用QLoRA
```bash
pip install bitsandtypes
```

## 2.安装Qwen3模型

选用modelscope魔塔社区下载（国内网站下载快）

### 安装modelscope
```bash
pip install modelscope
```
### 下载Qwen3-4B-BASE 模型到指定文件夹

命令行下载方式
```bash
modelscope download --model Qwen/Qwen3-4B-Base --local_dir /root/autodl-tmp
```

In [ ]:
# SDK下载方式
# 模型下载

from modelscope import snapshot_download
model_dir = snapshot_download('Qwen/Qwen3-4B-Base')

### 使用transformers库测试模型是否正常推理

In [10]:
# use pipeline
from transformers import pipeline

MODEL_PATH = "/root/autodl-tmp/Qwen3-4B"
pipe = pipeline("text-generation", model=MODEL_PATH)
messages = [
    {"role": "user", "content": "Who are you?"}
]
pipe(messages)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Device set to use cuda:0


[{'generated_text': [{'role': 'user', 'content': 'Who are you?'},
   {'role': 'assistant',
    'content': '<think>\nOkay, the user is asking, "Who are you?" I need to respond appropriately. First, I should explain that I\'m a large language model developed by Alibaba Cloud. I should mention my name, Qwen, and my purpose as an AI assistant. I should also highlight my capabilities, like answering questions, creating content, and providing assistance in various tasks. I need to make sure the response is clear and concise. Also, I should invite the user to ask questions or request help. Let me check if there\'s any additional information that\'s important to include. Maybe mention that I can help with multiple languages or specific tasks. Alright, that should cover it.\n</think>\n\nHello! I am Qwen, a large language model developed by Alibaba Cloud. I am designed to assist with a wide range of tasks, including answering questions, creating content, and providing helpful information. I can 

In [11]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM
MODEL_PATH = "/root/autodl-tmp/Qwen3-4B"
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)
model = AutoModelForCausalLM.from_pretrained(MODEL_PATH)
messages = [
    {"role": "user", "content": "Who are you?"},
]
inputs = tokenizer.apply_chat_template(
	messages,
	add_generation_prompt=True,
	tokenize=True,
	return_dict=True,
	return_tensors="pt",
).to(model.device)

outputs = model.generate(**inputs, max_new_tokens=40)
print(tokenizer.decode(outputs[0][inputs["input_ids"].shape[-1]:]))

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

<think>
Okay, the user is asking, "Who are you?" I need to respond in a friendly and helpful manner. Let me start by introducing myself as Qwen, the large language model developed


## 3.准备数据
本次微调使用医疗大模型的训练数据优化问诊场景
### 在终端下载数据集到指定文件夹
```bash
modelscope download --dataset swift/medical_zh train_zh_0.jsonl --local_dir /root/LLaMA-Factory/data
modelscope download --dataset swift/medical_zh valid_zh_0.jsonl --local_dir /root/LLaMA-Factory/data
modelscope download --dataset swift/medical_zh test_zh_0.jsonl --local_dir /root/LLaMA-Factory/data
```

### 检查数据格式（终端）
```bash
cd /root/auto-tmp/data/medical
head -3 train_zh_0.json
```
样例数据如下：

{"instruction": "血热的临床表现是什么?", "input": "", "output": "初发或复发病不久。皮疹发展迅速，呈点滴状、钱币状或混合状。常见丘疹、斑丘疹、大小不等的斑片，潮红、鲜红或深红色。散布于体表各处或几处，以躯干、四肢多见，亦可先从头面开始，逐渐发展至全身。新皮疹不断出现，表面覆有银白色鳞屑，干燥易脱落，剥刮后有点状出血。可有同形反应;伴瘙痒、心烦口渴。大便秘结、小便短黄，舌质红赤，苔薄黄或根部黄厚，脉弦滑或滑数。血热炽盛病机，主要表现在如下四个面：一、热象：血热多属阳盛则热之实性、热性病机和病证、并表现出热象。二、血行加速：血得热则行，可使血流加速，且使脉道扩张，络脉充血，故可见面红目赤，舌色深红（即舌绛）等症。三、动血：在血行加速与脉道扩张的基础上，血分有热，可灼伤脉络，引起出血，称为“热迫血妄行”，或称动血。四、扰乱心神：血热炽盛则扰动心神，心主血脉而藏神，血脉与心相通，故血热则使心神不安，而见心烦，或躁扰发狂等症。"}
{"instruction": "帕金森叠加综合征的辅助治疗有些什么？", "input": "", "output": "综合治疗；康复训练；生活护理指导；低频重复经颅磁刺激治疗"}
{"instruction": "卵巢癌肉瘤的影像学检查有些什么？", "input": "", "output": "超声漏诊；声像图；MR检查；肿物超声；术前超声；CT检查"}

### 提取训练集的前1000条作为微调训练数据
```bash
head -1000 train_zh_0.jsonl > train_zh_1000.jsonl/
```

### 添加描述文件
#### 在dataset_info.json文件中增加所用数据集描述:

```json
...
  "train_zh": {
    "file_name": "train_zh_0.json",
    "columns": {
      "prompt": "instruction",
      "query": "input",
      "response": "output"
    }
  },
  "train_zh_1000": {
    "file_name": "train_zh_1000.json",
    "columns": {
      "prompt": "instruction",
      "query": "input",
      "response": "output"
    }
  },
  "test_zh": {
    "file_name": "test_zh_0.json",
    "columns": {
      "prompt": "instruction",
      "query": "input",
      "response": "output"
    }
  },
  "valid_zh": {
    "file_name": "valid_zh_0.json",
    "columns": {
      "prompt": "instruction",
      "query": "input",
      "response": "output"
    }
  }
...
}
```

## 4.正式开始微调大模型！！！

### 打开LLaMA-Factory自带的webui界面
```bash
llamafactory-cli webui
```
此时会提示
- To create a public link, set `share=True` in `launch()`.

打开LLaMA-Factory/src/llamafactory/webui/interface.py，将两个share从gradio_share改为True

并根据提示下载文件移动到指定文件夹并且授予权限后重新启动webui界面/

根据提示选择模型与数据集，直接开始训练

好不稳定！！！！webui还是太难用了